In [1]:
import pandas as pd
import numpy as np
import operator
import datetime
import pyecharts
from eunjeon import Mecab 
#from konlpy.tag import Hannanum
from collections import Counter
from collections import defaultdict

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
videos = pd.read_csv('data/videos.csv')

In [3]:
videos['channel_title'].value_counts()[:50]

존크TV                        154
대고구려제국                      139
소녀통신 • 조회수239만회              80
Kraze Reaction               79
중국어로 [路]                     77
세상을말하다                       75
QUEEN SOHYANG                74
코리안브로스                       69
비더탑                          65
고춧가루TV                       63
ALL THE K-JAM 올더케이잼          61
국뽕                           54
에이 스토리 A STORY               49
구네스북                         49
뉴스킹                          47
해외리얼반응                       47
사이다 일본반응 • 조회수 73만회          44
레만 Reman [로마사는 한국남자]         43
BODA 보다                      42
사건왕-[해외반응, 외신뉴스]             41
tvN D ENT                    41
베이비버그 BabyBug                38
다른생각TV                       38
해외반응 탐사대                     37
월드 댓글TV                      36
올타구 · 시사 이슈 문화               36
이슈특급                         36
잇슈TV                         30
션데일리 리서치                     29
해외네티즌반응                      27
꾼맨 KKUNMAN                   27
팩튜브     

In [4]:
tagger = Mecab() 
vocabs = []
for title in videos['title']:
    nouns = tagger.nouns(title)
    for noun in nouns:
        vocabs.append(noun)

In [5]:
Counter(vocabs).most_common()

[('반응', 4261),
 ('한국', 3740),
 ('일본', 2176),
 ('해외', 1784),
 ('외국인', 979),
 ('리액션', 763),
 ('이유', 619),
 ('중국', 566),
 ('미국', 448),
 ('세계', 411),
 ('자막', 349),
 ('한글', 343),
 ('한국인', 323),
 ('일본인', 291),
 ('충격', 274),
 ('네티즌', 241),
 ('상황', 239),
 ('방탄소년단', 236),
 ('사람', 200),
 ('수', 169),
 ('방송', 158),
 ('영상', 155),
 ('모음', 151),
 ('처음', 149),
 ('거', 147),
 ('일', 135),
 ('외국', 133),
 ('말', 132),
 ('위', 131),
 ('영화', 127),
 ('드라마', 122),
 ('ㅣ', 122),
 ('커플', 115),
 ('소향', 114),
 ('대만', 110),
 ('나라', 107),
 ('것', 107),
 ('이것', 106),
 ('여자', 105),
 ('미국인', 104),
 ('년', 104),
 ('경악', 103),
 ('국제', 99),
 ('댓글', 99),
 ('친구', 98),
 ('영국', 97),
 ('최고', 94),
 ('미사일', 92),
 ('중', 89),
 ('남자', 88),
 ('비교', 87),
 ('먹방', 87),
 ('때', 86),
 ('현지', 84),
 ('우리', 84),
 ('탄', 83),
 ('ㄷ', 82),
 ('음식', 82),
 ('베트남', 82),
 ('만', 80),
 ('편', 80),
 ('사진', 79),
 ('문화', 79),
 ('대한민국', 79),
 ('폭발', 79),
 ('선진국', 79),
 ('뮤', 79),
 ('급', 78),
 ('무대', 78),
 ('레전드', 77),
 ('어로', 77),
 ('전', 76),
 ('군사력', 75),
 ('블랙

In [6]:
result = Counter(vocabs)

In [7]:
countries = pd.read_excel('data/countries.xlsx')

In [8]:
countries_name = list(countries['국가'])

In [9]:
countries_people = [ country + '인' for country in countries_name ]

In [10]:
countries_np = defaultdict()
for country in countries_name:
    if country in result.keys():
        countries_np[country] = result[country]
    if country + '인' in result.keys():
        countries_np[country] += result[country]

In [11]:
countries_np= sorted(countries_np.items(), key=operator.itemgetter(1), reverse=True)

In [12]:
videos['date'] = [ datetime.datetime.strptime(videos.loc[i, 'date'][:-13], '%Y-%m') for i in range(len(videos)) ]

In [36]:
videos.to_csv('videos.csv', encoding='utf-8-sig')

In [13]:
date_count = pd.DataFrame(videos['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]

In [14]:
from pyecharts import Bar, Line
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [15]:
tagger = Mecab() 

for i in range(len(videos)):
    countries_list = []
    nouns = tagger.nouns(videos.loc[i, 'title'])
    for noun in nouns:
        if noun in countries_name:
            countries_list.append(noun)
        if noun in countries_people:
            countries_list.append(noun[:-1])
    countries_list = list(set(countries_list))
    videos.loc[i, 'country'] = " ".join(countries_list)

In [37]:
videos.head(5)

,Unnamed: 0,title,video_id,channel_title,channel_id,description,date,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,country
0,0,한국 예능 야구 레전드를 보고 현웃터진 미국인 반응,AEl5RAzF0lQ,BODA 보다,UCoCvTlU0KpNYwnMIgs7MPrA,KBO #한국야구 #예능짤 #MLB 오늘은 미국에서 온 김태균씨와 한국에 레전드 야...,2020-07-01,62865.00,755.00,21.00,0.00,114.00,미국
1,1,해외에서 반응 난리 났다는 한국 디저트,wYp0kz2kgKE,뉴스킹,UClOg34aPlf3GWZZ6ojt0WYQ,"한국 #근황 #해외반응 해외에서 난리라는 한국 디저트 근황 ""돈 걱정 없이 전부 다...",2020-07-01,75606.00,1448.00,38.00,0.00,71.00,
2,2,&quot;한국 스타일 무섭네!&quot; 한국의 건물사진에 깜짝놀란 스페인 반응과...,It69uJHJIuA,구네스북,UClHJcrFMOxCEPjjg6WlTmPw,한국의 건물사진에 깜짝놀란 스페인 반응과 네티즌 반응을 다뤘습니다. * 구네스북 그...,2020-07-01,249430.00,5237.00,196.00,0.00,390.00,스페인
3,3,한국 LG OLED TV의인기비결은???해외반응 – 일본이것이 한국의 기술이다!!일...,1SIA3ukXyhc,데보이,UC9Uf0ZCFKKy65uCOf6oCrOQ,해외반응#한국해외반응#일본 한국 LG OLED TV의 인기비결은??? 해외반응 – ...,2020-07-01,232245.00,2439.00,119.00,0.00,148.00,일본
4,4,한국 마스크를 처음 써보고 놀라운 반응을 보여주는 이탈리아 남자! 그 이유는?,3LwSAs3eeQA,레만 Reman [로마사는 한국남자],UC0MhsY1Nc467re-66o6xwRg,이탈리아 남자 알레한테 한국 마스크를 선물해주었습니다. 알레 반응이 정말 대박이네요...,2020-07-01,43407.00,1455.00,20.00,0.00,65.00,이탈리아


In [38]:
countries_videocnt = []
countries_viewCount = []
countries_likeCount = []
countries_dislikeCount = []
countries_favoriteCount =[]
countries_commentCount = []


for country in countries_name:
    cnt = len(videos[videos['country'].str.contains(country)]['viewCount'])
    vc = videos[videos['country'].str.contains(country)]['viewCount'].mean()
    lc = videos[videos['country'].str.contains(country)]['likeCount'].mean()
    dc = videos[videos['country'].str.contains(country)]['dislikeCount'].mean()
    fc = videos[videos['country'].str.contains(country)]['favoriteCount'].mean()
    cc = videos[videos['country'].str.contains(country)]['commentCount'].mean()
    
    countries_videocnt.append(cnt)
    countries_viewCount.append(vc)
    countries_likeCount.append(lc)
    countries_dislikeCount.append(dc)
    countries_favoriteCount.append(fc)
    countries_commentCount.append(cc)

In [40]:
countires_stat = pd.DataFrame({'country':countries_name, 'video_num':countries_videocnt, 'viewCount':countries_viewCount,'likeCount':countries_likeCount, 'dislikeCount':countries_dislikeCount,
             'favoriteCount':countries_favoriteCount, 'commentCount': countries_commentCount})

In [41]:
countires_stat.to_csv('countries_stat.csv', encoding='utf-8-sig')

In [47]:
pd.options.display.float_format = '{:.2f}'.format
countires_stat[countires_stat['video_num']>=5].dropna().sort_values(by='commentCount', ascending=False)[:50]

,country,video_num,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
110,영국,96,764609.68,16328.23,430.38,0.00,1310.71
1,대만,62,393908.23,5352.35,190.02,0.00,894.02
23,몽골,6,407045.67,9419.17,294.50,0.00,865.50
10,미얀마,21,321833.43,7911.71,260.19,0.00,824.67
32,이스라엘,6,684348.67,12709.00,563.17,0.00,796.17
255,아시아,30,370455.00,4005.37,125.73,0.00,773.03
25,중국,317,322566.91,5733.24,190.64,0.00,753.71
157,러시아,54,440102.30,11370.52,182.44,0.00,749.06
30,이란,8,379562.75,6918.00,117.75,0.00,514.50
26,북한,24,315490.48,6485.48,140.00,0.00,487.35


In [100]:
df_america = videos[videos['country'].str.contains('미국')]
date_count = pd.DataFrame(df_america['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [101]:
df_japan = df[df['country'].str.contains('일본')]
date_count = pd.DataFrame(df_japan['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [102]:
df_china = df[df['country'].str.contains('중국')]
date_count = pd.DataFrame(df_china['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [104]:
df_china = df[df['country'].str.contains('스페인')]
date_count = pd.DataFrame(df_china['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [105]:
df_china = df[df['country'].str.contains('이탈리아')]
date_count = pd.DataFrame(df_china['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [108]:
df_china = df[df['country'].str.contains('베트남')]
date_count = pd.DataFrame(df_china['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar

In [109]:
df_china = df[df['country'].str.contains('싱가포르')]
date_count = pd.DataFrame(df_china['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar